In [ ]:
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 4.2 MB/s 


In [ ]:
# Import some libraries

import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
seed = 0
np.random.seed(seed)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'whitegrid')

import nest_asyncio
nest_asyncio.apply()

import datetime as dt
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, LSTM
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

In [ ]:
#access google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#rename nama kolom
df = pd.read_csv('/content/drive/My Drive/dikumpulkan/Invasi_Rusia_file.csv')
df.rename({'Unnamed: 0':'no'}, axis=1, inplace = True)
df.head(10)

In [ ]:
# Load data from a CSV file into pandas DataFrame
tweets_data = pd.read_csv('/content/drive/My Drive/data/Invasi_Rusia_file.csv')
tweets = tweets_data[['created_at','text']]
tweets.head(20)

In [ ]:
# Some functions for preprocessing text

# Initializing stemming library
factory = StemmerFactory()
stemmer = factory.create_stemmer()

from urllib.request import urlopen
import json

def cleaningText(text):#function tahap cleaning
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # remove mentions
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # remove hashtag
    text = re.sub(r'RT[\s]', '', text) # remove RT
    text = re.sub(r"http\S+", '', text) # remove link
                                        #text = re.sub(r'[0-9]+', '', text) # remove numbers
    text = re.sub(r'[^A-Za-z ]+', '', text) #remove all character non alfabet

    text = text.replace('\n', ' ') # replace new line into space
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove all punctuations
    text = text.strip(' ') # remove characters space from both left and right text
    return text

def casefoldingText(text): # Converting all the characters in a text into lower case
    text = text.lower() 
    return text

def tokenizingText(text): # Tokenizing or splitting a string, text into a list of tokens
    text = word_tokenize(text)
    return text

def filteringText(text): # Remove stopwors in a text
    listStopwords = set(stopwords.words('indonesian'))
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered 
    return text

def stemmingText(text): # Reducing a word to its word stem that affixes to suffixes and prefixes or to the roots of words
    text = [stemmer.stem(word) for word in text]
    return text

def toSentence(list_words): # Convert list of words into sentence
    sentence = ' '.join(word for word in list_words)
    return sentence

# slang indonesia
response_slang = urlopen('https://raw.githubusercontent.com/louisowen6/NLP_bahasa_resources/master/combined_slang_words.txt')
data_slang = json.loads(response_slang.read().decode())
def slangWordID(t):
    t = " ".join([data_slang.get(w,w) for w in t.split()])
    return t

# print(slangWordID("ka"))

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
# tweets = tweets[:5]

# Preprocessing tweets data
tweets['text_clean'] = tweets['text'].apply(cleaningText)
tweets.head(20)

In [ ]:
tweets['text_clean'] = tweets['text_clean'].apply(casefoldingText)
# tweets.drop(['text'], axis = 1, inplace = True)
tweets.head(30)

In [ ]:
tweets['text_preprocessed'] = tweets['text_clean'].apply(tokenizingText)
tweets.head(30)

In [ ]:
tweets['text_preprocessed'] = tweets['text_preprocessed'].apply(filteringText)
tweets.head(30)

In [ ]:
tweets['text_preprocessed'] = tweets['text_preprocessed'].apply(stemmingText)
tweets.head(30)

In [ ]:
tweets

In [ ]:
tweets['text_preprocessed'] = tweets['text_preprocessed'].apply(toSentence)
tweets.head(10)

In [ ]:
tweets['text_preprocessed'] = tweets['text_preprocessed'].apply(slangWordID)

# drop duplicates/spams tweets
tweets.drop_duplicates(subset = 'text_clean', inplace = True)
tweets.head(50)

In [ ]:
# Export to csv file
tweets.to_csv(r'/content/drive/My Drive/dikumpulkan/preposecessing.csv', index = False, header = True,index_label=None)

tweets

# print(tweets['text_clean'][0])
# print('\r')
# print(tweets['text_preprocessed'][0])